In [1]:
!nvidia-smi

Sun Sep 21 08:07:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   32C    P8             14W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|  No running processes found                                                     

In [2]:
!pip install git+https://github.com/RyanWangZf/MedCLIP.git
!pip install pyarrow==14.0.2 datasets==2.16.1

  Cloning https://github.com/RyanWangZf/MedCLIP.git to /tmp/pip-req-build-y8i17j9e
  Running command git clone --filter=blob:none --quiet https://github.com/RyanWangZf/MedCLIP.git /tmp/pip-req-build-y8i17j9e


  Resolved https://github.com/RyanWangZf/MedCLIP.git to commit 9c3396f20d5d54e4fae241b8cb06ca45848e98c9
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install kagglehub datasets


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os
import pandas as pd

In [5]:
%cd MedCLIP

/workspace/jupyter_notebooks/data_parquet/MedCLIP


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Chexpert

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]

# # Download the dataset and explore available files
# print("Downloading CheXpert dataset...")
# dataset_path = kagglehub.dataset_download('ashery/chexpert')
# print(f"Dataset downloaded to: {dataset_path}")

chexpert_val_csv = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "ashery/chexpert",
    "valid.csv",
)

/tmp/ipykernel_6841/1710688056.py:12: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  chexpert_val_csv = kagglehub.load_dataset(


In [ ]:
def create_chexpert_5x200(chexpert_val_csv):

    condition_columns = ['Atelectasis', 'Cardiomegaly', 'Edema', 'Pleural Effusion', 'Consolidation']

    other_cols = [col for col in condition_columns if col != 'Atelectasis']
    unique_Atelectasis = chexpert_val_csv[(chexpert_val_csv['Atelectasis'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Cardiomegaly']
    unique_Cardiomegaly = chexpert_val_csv[(chexpert_val_csv['Cardiomegaly'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Edema']
    unique_Edema = chexpert_val_csv[(chexpert_val_csv['Edema'] != 0) & 
                            (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Pleural Effusion']
    unique_PleuralEffusion = chexpert_val_csv[(chexpert_val_csv['Pleural Effusion'] != 0) & 
                                        (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Consolidation']
    unique_Consolidation = chexpert_val_csv[(chexpert_val_csv['Consolidation'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]
    return unique_Atelectasis, unique_Cardiomegaly, unique_Edema, unique_PleuralEffusion, unique_Consolidation

# MIMIC CXR

In [10]:
from datasets import load_dataset

ds = load_dataset("itsanmolgupta/mimic-cxr-dataset")

Generating train split:   0%|          | 0/30633 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'findings', 'impression'],
        num_rows: 30633
    })
})

# COVID 19

In [28]:
%cd MedCLIP

/workspace/jupyter_notebooks/data_parquet/MedCLIP


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Prepare dataset

In [7]:
covid_data_path = kagglehub.dataset_download('tawsifurrahman/covid19-radiography-database')

100%|██████████| 778M/778M [00:36<00:00, 22.6MB/s] 

Extracting files...


In [57]:
def create_covid_csv(covid_lst, non_covid_lst):
    df = pd.DataFrame(columns=['imgpath', 'COVID', 'Normal'])
    for covid in covid_lst:
        new_row = pd.DataFrame({'imgpath': [covid], 'COVID': [1], 'Normal': [0]})
        df = pd.concat([df, new_row], ignore_index=True)
    for non_covid in non_covid_lst:
        new_row = pd.DataFrame({'imgpath': [non_covid], 'COVID': [0], 'Normal': [1]})
        df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv('local_data/covid-test-meta.csv', index=True)
    return df

In [11]:
covid_prompt = "the presence of patchy or confluent, bandlike ground-glass opacity or consolidation in a peripheral and mid to lower lung zone distribution"

In [12]:
covid_data_path = os.path.join(covid_data_path, 'COVID-19_Radiography_Dataset')

In [13]:
covid_data_path

'/workspace/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset'

In [14]:
os.listdir(
    covid_data_path
)

['Lung_Opacity.metadata.xlsx',
 'Viral Pneumonia',
 'Viral Pneumonia.metadata.xlsx',
 'Lung_Opacity',
 'COVID.metadata.xlsx',
 'Normal',
 'COVID',
 'Normal.metadata.xlsx',
 'README.md.txt']

In [18]:
viral_pneumonia_path = os.path.join(
        covid_data_path,
        'Viral Pneumonia',
        'images'
)
viral_pneumonia_path_list = [os.path.join(viral_pneumonia_path, f) 
                            for f in os.listdir(viral_pneumonia_path)]

lung_opacity_path = os.path.join(
        covid_data_path,
        'Lung_Opacity',
        'images'
)
lung_opacity_path_list = [os.path.join(lung_opacity_path, f)
                         for f in os.listdir(lung_opacity_path)]

normal_path = os.path.join( 
        covid_data_path,
        'Normal',
        'images'
)
normal_path_list = [os.path.join(normal_path, f)
                   for f in os.listdir(normal_path)]

covid_path = os.path.join(
        covid_data_path,
        'COVID',
        'images'
)
covid_path_list = [os.path.join(covid_path, f)
                  for f in os.listdir(covid_path)]

In [58]:
covid_csv = create_covid_csv(covid_path_list, 
                            viral_pneumonia_path_list + lung_opacity_path_list + normal_path_list)

## Zero shot classification

In [ ]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, PromptClassifier
from medclip.dataset import ZeroShotImageDataset, ZeroShotImageCollator
from medclip.evaluator import Evaluator
from medclip.prompts import generate_covid_class_prompts
from medclip.modeling_medclip import PromptClassifier
from torch.utils.data import DataLoader

In [ ]:
# Generate COVID class prompts
covid_prompts = generate_covid_class_prompts(n=5)
print("COVID class prompts:", covid_prompts)


sample 5 num of prompts for COVID from total 12
COVID class prompts: {'COVID': ['patchy ground glass consolidation in mid', 'confluent ground glass opacity in lower', 'confluent ground glass consolidation in mid', 'patchy ground glass opacity in lower', 'patchy ground glass consolidation in peripheral']}


In [ ]:
class_names = ['COVID', 'Normal']

covid_clf_dataset = ZeroShotImageDataset(
    datalist=['covid-test'],
    class_names=class_names,
    imgtransform=None
)

# Create the collator with COVID prompts
covid_collator = ZeroShotImageCollator(
    mode='binary',
    cls_prompts=covid_prompts,
    n_prompt=5
)

covid_dataloader= DataLoader(
    covid_clf_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,  # Set to 0 to avoid multiprocessing issues
    collate_fn=covid_collator
)

resnet_clf = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
resnet_clf.from_pretrained()
resnet_clf.cuda()

covid_classifier = PromptClassifier(
    medclip_model=resnet_clf,
    ensemble=True  
)

evaluator = Evaluator(
    medclip_clf=covid_classifier,
    eval_dataloader=covid_dataloader_fixed,
    mode='binary'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializi

load model weight from: ./pretrained/medclip-vit


In [ ]:
print("Testing the dataloader...")
test_batch = next(iter(covid_dataloader))
print("Batch keys:", test_batch.keys())
print("Pixel values shape:", test_batch['pixel_values'].shape)
print("Labels shape:", test_batch['labels'].shape)
print("Prompt inputs keys:", test_batch['prompt_inputs'].keys())

Testing the dataloader...
Batch keys: dict_keys(['pixel_values', 'prompt_inputs', 'labels'])
Pixel values shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Prompt inputs keys: dict_keys(['COVID'])


In [ ]:
print("\nRunning evaluation...")
results = evaluator.evaluate()


Running full evaluation...


Evaluation: 100%|██████████| 662/662 [03:24<00:00,  3.24it/s]

Evaluation results: {'pred': array([[0.6567303 ],
       [0.5656682 ],
       [0.71994305],
       ...,
       [0.1561554 ],
       [0.21197644],
       [0.20313087]], dtype=float32), 'labels': array([0, 0, 0, ..., 1, 1, 1]), 'auc': 0.3268218378552695, 'acc': 0.8291519017245452, 'precision': 0.4145759508622726, 'recall': 0.5, 'f1-score': 0.4532985483287699}



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
print(f"Results: {results}")
print(f"accuracy: {results['acc']}")
print(f"precision: {results['precision']}")
print(f"recall: {results['recall']}")
print(f"f1_score: {results['f1-score']}")


Results: {'pred': array([[0.6567303 ],
       [0.5656682 ],
       [0.71994305],
       ...,
       [0.1561554 ],
       [0.21197644],
       [0.20313087]], dtype=float32), 'labels': array([0, 0, 0, ..., 1, 1, 1]), 'auc': 0.3268218378552695, 'acc': 0.8291519017245452, 'precision': 0.4145759508622726, 'recall': 0.5, 'f1-score': 0.4532985483287699}
accuracy: 0.8291519017245452
precision: 0.4145759508622726
recall: 0.5
f1_score: 0.4532985483287699


# RNSA 

## Prepare dataset

In [6]:
!pip install gdown pydicom


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import gdown
import zipfile

rsna_path = "rsna_data"

def download_rsna_dataset():    
    url_id = "1r2mIaytuvHQc5D77BAuIfkLeW9jGXjed"
    rsna_output = "rsna_pneumonia_detection.zip"
    print("Downloading RSNA dataset from Google Drive...")
    gdown.download(id=url_id, output=rsna_output, quiet=False)
    with zipfile.ZipFile(rsna_output, 'r') as zip_ref:
        zip_ref.extractall("./rsna_data")

In [8]:
if os.path.exists(rsna_path):
    print("RSNA dataset already downloaded and extracted successfully!")
else:
    download_rsna_dataset()
    print("RSNA dataset downloaded and extracted successfully!")

RSNA dataset already downloaded and extracted successfully!


In [9]:
def create_rsna_csv(pneumonia_lst, normal_lst):
    df = pd.DataFrame(columns=['imgpath', 'Pneumonia', 'Normal'])
    for patient_id in pneumonia_lst:
        new_row = pd.DataFrame({'imgpath': [patient_id], 'Pneumonia': [1], 'Normal': [0]})
        df = pd.concat([df, new_row], ignore_index=True)
    for patient_id in normal_lst:
        new_row = pd.DataFrame({'imgpath': [patient_id], 'Pneumonia': [0], 'Normal': [1]})
        df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv('local_data/rsna-balanced-test-meta.csv', index=True)
    return df

In [10]:
img_folder_path = "/workspace/jupyter_notebooks/data_parquet/MedCLIP/rsna_data/stage_2_train_images"
train_csv = pd.read_csv("/workspace/jupyter_notebooks/data_parquet/MedCLIP/rsna_data/stage_2_train_labels.csv")
pneumonia_lst = []
normal_lst = []
for index, row in train_csv.iterrows():
    if row['Target'] == 1:
        pneumonia_lst.append(os.path.join(img_folder_path, f"{row['patientId']}.dcm"))
    else:
        normal_lst.append(os.path.join(img_folder_path, f"{row['patientId']}.dcm"))


In [11]:
len(pd.read_csv("/workspace/jupyter_notebooks/data_parquet/MedCLIP/rsna_data/stage_2_train_labels.csv")), len(pd.read_csv("/workspace/jupyter_notebooks/data_parquet/MedCLIP/rsna_data/stage_2_detailed_class_info.csv"))

(30227, 30227)

In [12]:
create_rsna_csv(pneumonia_lst, normal_lst)

,imgpath,Pneumonia,Normal
0,/workspace/jupyter_notebooks/data_parquet/MedC...,1,0
1,/workspace/jupyter_notebooks/data_parquet/MedC...,1,0
2,/workspace/jupyter_notebooks/data_parquet/MedC...,1,0
3,/workspace/jupyter_notebooks/data_parquet/MedC...,1,0
4,/workspace/jupyter_notebooks/data_parquet/MedC...,1,0
...,...,...,...
30222,/workspace/jupyter_notebooks/data_parquet/MedC...,0,1
30223,/workspace/jupyter_notebooks/data_parquet/MedC...,0,1
30224,/workspace/jupyter_notebooks/data_parquet/MedC...,0,1
30225,/workspace/jupyter_notebooks/data_parquet/MedC...,0,1


## Zero shot classification

In [13]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, PromptClassifier
from medclip.dataset import ZeroShotImageDataset, ZeroShotImageCollator
from medclip.evaluator import Evaluator
from medclip.prompts import generate_rsna_class_prompts
from medclip.modeling_medclip import PromptClassifier
from torch.utils.data import DataLoader

In [14]:
# Generate RSNA class prompts
rsna_prompts = generate_rsna_class_prompts(n=5)
print("RSNA class prompts:", rsna_prompts)

sample 5 num of prompts for Pneumonia from total 312
RSNA class prompts: {'Pneumonia': ['early  ', 'multifocal mycoplasma at the left lower lobe', 'early viral at the left lung zone', ' viral at the left middle lobe', 'focal  at the right lung base']}


In [16]:
class_names = ['Pneumonia', 'Normal']

# Use the DICOM-compatible dataset class for RSNA data
rsna_clf_dataset = ZeroShotImageDataset(
    datalist=['rsna-balanced-test'],
    class_names=class_names,
    imgtransform=None
)

# Create the collator with RSNA prompts
rsna_collator = ZeroShotImageCollator(
    mode='binary',
    cls_prompts=rsna_prompts,
    n_prompt=5
)

rsna_dataloader= DataLoader(
    rsna_clf_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,  # Set to 0 to avoid multiprocessing issues
    collate_fn=rsna_collator
)

resnet_clf = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
resnet_clf.from_pretrained()
resnet_clf.cuda()

rsna_classifier = PromptClassifier(
    medclip_model=resnet_clf,
    ensemble=True  
)

evaluator = Evaluator(
    medclip_clf=rsna_classifier,
    eval_dataloader=rsna_dataloader,
    mode='binary'
)

load data from ./local_data/rsna-balanced-test-meta.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This I

load model weight from: ./pretrained/medclip-vit


In [17]:
print("Testing the dataloader...")
test_batch = next(iter(rsna_dataloader))
print("Batch keys:", test_batch.keys())
print("Pixel values shape:", test_batch['pixel_values'].shape)
print("Labels shape:", test_batch['labels'].shape)
print("Prompt inputs keys:", test_batch['prompt_inputs'].keys())

Testing the dataloader...
Batch keys: dict_keys(['pixel_values', 'prompt_inputs', 'labels'])
Pixel values shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Prompt inputs keys: dict_keys(['Pneumonia'])


In [18]:
# Run RSNA evaluation
print("\nRunning RSNA evaluation...")
try:
    results = evaluator.evaluate()
    print(f"Results: {results}")
    print(f"accuracy: {results['acc']}")
    print(f"precision: {results['precision']}")
    print(f"recall: {results['recall']}")
    print(f"f1_score: {results['f1-score']}")
except Exception as e:
    print(f"Evaluation failed with error: {e}")
    import traceback
    traceback.print_exc()



Running RSNA evaluation with DICOM support...


Evaluation: 100%|██████████| 945/945 [09:04<00:00,  1.74it/s]

Results: {'pred': array([[0.41325304],
       [0.28632236],
       [0.39889213],
       ...,
       [0.4238336 ],
       [0.36507756],
       [0.39240953]], dtype=float32), 'labels': array([1, 1, 1, ..., 1, 1, 0]), 'auc': 0.4722937606216575, 'acc': 0.6838918847388097, 'precision': 0.34194594236940484, 'recall': 0.5, 'f1-score': 0.40613764514037604}
accuracy: 0.6838918847388097
precision: 0.34194594236940484
recall: 0.5
f1_score: 0.40613764514037604



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
